In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nlppln
import ochre
import os

working_dir = '/home/jvdzwaan/cwl-working-dir/'

with nlppln.WorkflowGenerator(working_dir=working_dir) as wf:
    wf.load(steps_dir=ochre.cwl_path())
    print(wf.list_steps())

    archive = wf.add_input(archive='File')
    ocr_dir_name = wf.add_input(ocr_dir_name='string?', default='ocr')
    gs_dir_name = wf.add_input(gs_dir_name='string?', default='gs')
    
    in_dir = wf.archive2dir(archive=archive)

    vudnc_files = wf.vudnc_select_files(in_dir=in_dir)
    gs_with_empty, ocr_with_empty = wf.vudnc2ocr_and_gs(in_file=vudnc_files, 
                                                        scatter='in_file', scatter_method='dotproduct')
    gs_dir = wf.save_files_to_dir(dir_name=gs_dir_name, in_files=gs_with_empty)
    ocr_dir = wf.save_files_to_dir(dir_name=ocr_dir_name, in_files=ocr_with_empty)
    gs, ocr = wf.remove_empty_files(gs_dir=gs_dir, ocr_dir=ocr_dir)

    gs_dir = wf.save_files_to_dir(dir_name=gs_dir_name, in_files=gs)
    ocr_dir = wf.save_files_to_dir(dir_name=ocr_dir_name, in_files=ocr)

    wf.add_outputs(gs_dir=gs_dir, ocr_dir=ocr_dir)
    wf.save(os.path.join(ochre.cwl_path(), 'vudnc-preprocess-pack.cwl'), pack=True, relative=False)